# 全部采用本地化部署的方式和开源模型来建立的问答能力和本地化知识库集成的能力

定义语言模型

In [1]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()
model = model.eval()

/home/xiaodong/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No compiled kernel found.
Compiling kernels : /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.c -shared -o /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.so
Load kernel : /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 6
Using quantization cache
Applying quantization to glm layers


按照langchain标准进行模型封装

In [6]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
from llama_index import LLMPredictor

class CustomLLM(LLM):

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response, history = model.chat(tokenizer, prompt, history=[])
        # only return newly generated tokens
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": "THUDM/chatglm-6b-int4"}

    @property
    def _llm_type(self) -> str:
        return "custom"
    
llm_predictor = LLMPredictor(llm=CustomLLM())



引入embedding模型

In [7]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))

定义包含embedding模型和语言模型的Service Context

In [8]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm_predictor=llm_predictor)

构建本地知识库（本地知识的索引库）

In [9]:
from langchain.text_splitter import CharacterTextSplitter
from llama_index import SimpleDirectoryReader
from llama_index import GPTVectorStoreIndex

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=80, chunk_overlap=20)

documents = SimpleDirectoryReader('./data/faq/').load_data()


index = GPTVectorStoreIndex.from_documents(documents=documents, service_context=service_context)
query_engine = index.as_query_engine()

结合本地知识库和语言模型进行提问

In [10]:
question = """
请用中文回答下述问题：

Q: 你们能配送到天津吗？另外，你们的退货政策是怎样的？
"""
response = query_engine.query(question)
print(response)

The dtype of attention mask (torch.int64) is not bool
Token indices sequence length is longer than the specified maximum sequence length for this model (1878 > 1024). Running this sequence through the model will result in indexing errors


A: 我们的物流公司可以配送到天津。关于退货政策，我们在订单发货后的24小时内提供在线客服支持，方便用户联系处理退货事宜。具体的退货政策请参考订单页面或联系客服咨询。


In [11]:
question = """
请用中文回答下述问题：

Q: 请介绍一下ChatGLM？
"""
response = query_engine.query(question)
print(response)

ChatGLM-6B 是由清华大学 KEG 实验室和智谱 AI 公司于 2023年 共同开发的人工智能助手。它是基于 GLM-6B 模型开发的，能够针对用户的问题和要求提供适当的答复和支持。


In [13]:
question = """
请用中文回答下述问题：

Q: 请介绍一下语言模型中的Alpaca。
"""
response = query_engine.query(question)
print(response)

Alpaca是一个开源的语言模型，由清华大学 KEG 实验室和智谱AI开发。它是基于 GLM-130B 模型开发的，拥有 1300 亿个参数，是GLM-130B 的 10 倍。Alpaca 可以帮助人类完成一些语言任务，如文本生成、翻译、问答等。它可以通过自然语言处理技术，对文本进行建模，并生成与输入文本相似的自然语言输出。Alpaca 可以在各种平台上使用，包括 Python、TensorFlow、PyTorch 等。


In [14]:
question = """
请用中文回答下述问题：

Q: 请写一段python代码，展示如何使用Alpaca的模型在本地GPU运行支持语言任务的能力。
"""
response = query_engine.query(question)
print(response)

以下是一段Python代码，展示如何使用Alpaca的模型在本地GPU运行支持语言任务的能力：

```python
import tensorflow as tf

# 加载模型
model = tf.keras.models.load_model('model.json')

# 定义训练函数
def train_model(x_train, y_train, batch_size, epochs):
    # 定义损失函数
    def loss_fn(x, y):
        return tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(y_train > 0)

    # 定义优化函数
    def optimize_model(model, x_train, y_train, batch_size, epochs):
        # 计算损失函数
        loss = loss_fn(model.predict(x_train), y_train)

        # 计算梯度
        梯度 = model.trainable_variables[0] * loss

        # 更新模型参数
        model.trainable_variables[0] +=梯度

        # 计算反向传播
        反向传播 = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_train, logits=model.predict(x_train)))

        # 计算优化目标
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

        # 训练模型
        model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
        model.fit(x_train, y_train, epochs=epochs)

# 运行训练函数
train_m